# Deep Learning


**TODO**

1. ~Ho to define models~ 

    * ~Helow word simple regression~
    * ~define models with sequence~
    
1. Callbacks 
    * How to stop when reach specific acc or perfromance 
    * How to save best model during 
    * How to save models archicterue, parameters, optmizer parameters (everything) (see my code in dialects)
    * How to load saved models 

1. ~Vectorization operations and loss functions~

    * ~create ones and zeros and get shape~
    * ~matrix n vector mutliplications~
    * ~reshape matrix~
    * ~How to encode one-hot~
    
1. Code for main loss functions and when to use

    * binnary cross entropy
    * categorical cross entropy
    * sparse categorical cross entropy
    * mae
    * mse 

1. Learning curves 
    * Code to plot learning curves in notebook using matplotlib
    * Code to use tensorboard. Add how to use and interpret the histogram of weight intensorboard

1. Code to use Genrators
    * Split folders in train and val so Generators can read 
    * ImageDataGenerator with data augmentation
    * Use ImageDataGenerator to save aigmented data

refs:
* https://blog.goodaudience.com/artificial-neural-networks-explained-436fcf36e75


## Tensorflow 


In [2]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
#from tensorflow.python.framework import ops
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

#slim = tf.contrib.slim

%matplotlib inline
np.random.seed(1)

print(tf.__version__)

2.0.0


## Hello word (regression)

In [72]:
# the data
# rule that We need to find is y = 2x - 1
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0,1.0,3.0,5.0,7.0], dtype=float)

# Define a model with one neuron
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])

# define loss func and optimizer 
model.compile(optimizer='sgd', loss='mean_squared_error')

# train or learn the rule
model.fit(xs, ys, epochs=500, verbose=0)

print("Checking")
x = 3.0
print(f"x: {x}; expected: {2.0*x-1}; prediction: {model.predict([x])}")

print("unseen data")
x = 10.00
print(f"x: {x}; expected: {2.0*x-1}; prediction: {model.predict([x])}")

Checking
x: 3.0; expected: 5.0; prediction: [[5.000253]]
unseen data
x: 10.0; expected: 19.0; prediction: [[18.982594]]


## Tensor basic operations 

https://www.tensorflow.org/guide/tensor


In [55]:
import numpy as np

print(f"vector add: {tf.add([1, 2], [3, 4])}")
print(f"sum all elemenst: {tf.reduce_sum([1, 2, 3])}")

mymat = tf.Variable([[7],[11]], tf.int16)
print(f"mymat: {mymat.value()}")

# batch x height x width x color
batch_imgs = tf.zeros([10, 299, 299, 3])  
print(f"batch_imgs: {batch_imgs.shape}")

print()

y_hat = tf.constant(36)            # Define y_hat constant. Set to 36.
y = tf.constant(39)   

loss = tf.Variable((y - y_hat)**2)

print(f"loss: {loss.value()}")

print()
print("numpy compatibility")

ndarray = np.ones([3, 3])

print("Tensor convert to numpy")
tensor = tf.multiply(ndarray, 10)
print(f"tensor: {tensor}")

print()
print("And Tensors converts to numpy")
print(f" add scalr to tensor and get a numpy array:  {np.add(tensor, 1)}")

print()
print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

print()
print(f"tensor ones: {tf.Variable(np.ones([3, 3]))}")

print()
print("Reshaping")

x = tf.reshape(np.array([1,2,3,4,5,6,7,8,9]), [3, 3])
print(f"x shape: {x.shape}")
print(f"x: {x.numpy()}")

# -1 means infere this dimension
y = tf.reshape(np.array([1,2,3,4,5,6,7,8,9,10,11,12]), [2, -1])
print(f"y shape: {y.shape}")
print(f"x: {y.numpy()}")

vector add: [4 6]
sum all elemenst: 6
mymat: [[ 7]
 [11]]
batch_imgs: (10, 299, 299, 3)

loss: 9

numpy compatibility
Tensor convert to numpy
tensor: [[10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]]

And Tensors converts to numpy
 add scalr to tensor and get a numpy array:  [[11. 11. 11.]
 [11. 11. 11.]
 [11. 11. 11.]]

The .numpy() method explicitly converts a Tensor to a numpy array
[[10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]]

tensor ones: <tf.Variable 'Variable:0' shape=(3, 3) dtype=float64, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])>

Reshaping
x shape: (3, 3)
x: [[1 2 3]
 [4 5 6]
 [7 8 9]]
y shape: (2, 6)
x: [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]]


### Encoding

* One hot encoding

https://www.tensorflow.org/api_docs/python/tf/one_hot

<img src="images/onehot.png" width="800" align="left"/>

In [48]:
from tensorflow.keras.utils import to_categorical

test_labels = tf.constant([1,2, 3, 0, 2, 1])

y = to_categorical(test_labels)
y.shape

k = 1
print(f"labels: {test_labels[k]}; one-hot encode: {y[k,:]}")
y

labels: 2; one-hot encode: [0. 0. 1. 0.]


array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [49]:
indices = [0, 1, 2]
depth = 3

# ?tf.one_hot
tf.one_hot(indices, depth)  # output: [3 x 3]

<tf.Tensor: id=825, shape=(3, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)>

### Train 


- **Training set**: 1080 pictures (64 by 64 pixels) of signs representing numbers from 0 to 5 (180 pictures per number).
- **Test set**: 120 pictures (64 by 64 pixels) of signs representing numbers from 0 to 5 (20 pictures per number).


In [ ]:
def load_dataset():
    
    train_dataset = h5py.File('data/train_signs.h5', "r")
    
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('data/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

In [ ]:
print(np.shape(classes))
print(classes)

# See example of image
index = 0
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

* preprocessing the images

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

# Flatten the training and test images
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T

# Normalize image vectors
X_train = X_train_flatten / 255.
X_test = X_test_flatten / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6)
Y_test = convert_to_one_hot(Y_test_orig, 6)

print("number of training examples = " + str(X_train.shape[1]))
print("number of test examples = " + str(X_test.shape[1]))

print()
print('64*64*3 = 12288')
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

print()
print ("Y_test (1st 5) = \n" + str(np.squeeze(Y_test[:,0:5])))



```python

# Use the loss function (approx. 1 line)
cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=z, labels=y)

```

$$- \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log \sigma(z^{[2](i)}) + (1-y^{(i)})\log (1-\sigma(z^{[2](i)})\large )\small\tag{2}$$




In [ ]:
def model_network():
    
    with tf.variable_scope('mymodel'):
        
    # Add a fully connected layer with 100 units.
    num_units = 100
    fc = slim.fully_connected(embeddings, num_units)

    # Add a classifier layer at the end, consisting of parallel logistic
    # classifiers, one per class. This allows for multi-class tasks.
    logits = slim.fully_connected( fc, _NUM_CLASSES, activation_fn=None, scope='logits')
        
    tf.sigmoid(logits, name='prediction')


with tf.Graph().as_default(), tf.Session() as sess:
    
    

### Prediction or inference

## Tensorflow slim

**TF-Slim: A high level library to define complex models in TensorFlow**

refs:

https://cv-tricks.com/tensorflow-tutorial/understanding-alexnet-resnet-squeezenetand-running-on-tensorflow/

https://github.com/tensorflow/models/blob/master/research/slim/slim_walkthrough.ipynb


**Comparing slim with tensorflow. You need less lines of code to do the same**. But tensorflow gives you more flexibility and it is easy to see the learning curves

```python
# using slim
net = slim.conv2d(input, 128,[3, 3], scope='conv1_1')

# slim version
with tf.name_scope('conv1_1') as scope:
    
    kernel = tf.Variable(tf.truncated_normal([3, 3,  64,128], dtype=tf.float32,stddev=1e-1), name='weights')
    conv = tf.nn.conv2d(input, kernel,[1, 1, 1, 1], padding='SAME')
 
    biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32), trainable=True, name='biases')
    
    bias = tf.nn.bias_add(conv, biases)
 
    conv1 = tf.nn.relu(bias, name=scope)
     
```

### Regression 

#### Define model network

In [ ]:
def regression_model(inputs, is_training=True, scope="deep_regression"):
    """Creates the regression model.

    Args:
        inputs: A node that yields a `Tensor` of size [batch_size, dimensions].
        is_training: Whether or not we're currently training the model.
        scope: An optional variable_op scope for the model.

    Returns:
        predictions: 1-D `Tensor` of shape [batch_size] of responses.
        end_points: A dict of end points representing the hidden layers.
    """
    with tf.variable_scope(scope, 'deep_regression', [inputs]):
        
        end_points = {}
        
        # Set the default weight _regularizer and acvitation for each fully_connected layer.
        # To set defaults parameters for a layer type
        with slim.arg_scope([slim.fully_connected],
                            activation_fn=tf.nn.relu,
                            weights_regularizer=slim.l2_regularizer(0.01)):

            # Creates a fully connected layer from the inputs with 32 hidden units.
            net = slim.fully_connected(inputs, 32, scope='fc1')
            end_points['fc1'] = net

            # Adds a dropout layer to prevent over-fitting.
            net = slim.dropout(net, 0.8, is_training=is_training)

            # Adds another fully connected layer with 16 hidden units.
            net = slim.fully_connected(net, 16, scope='fc2')
            end_points['fc2'] = net

            # Creates a fully-connected layer with a single hidden unit. Note that the
            # layer is made linear by setting activation_fn=None.
            predictions = slim.fully_connected(net, 1, activation_fn=None, scope='prediction')
            end_points['out'] = predictions

            return predictions, end_points

        
# Underatnd the model
with tf.Graph().as_default():
    
    # Dummy placeholders for arbitrary number of 1d inputs and outputs
    inputs = tf.placeholder(tf.float32, shape=(None, 1))
    #outputs = tf.placeholder(tf.float32, shape=(None, 1))

    # Build model
    predictions, end_points = regression_model(inputs)

    # Print name and shape of each tensor.
    print("Layers")
    for k, v in end_points.items():
        print('name = {}, shape = {}'.format(v.name, v.get_shape()))

    # Print name and shape of parameter nodes  (values not yet initialized)
    print("\n")
    print("Parameters")
    for v in slim.get_model_variables():
        print('name = {}, shape = {}'.format(v.name, v.get_shape()))

#### Train


##### Using slim.learning.train

**Use slim training procedure is hard to get the learning curves and evaluations**. Its being criticized by the comunity. Might be why Google use slim to build the mode and for loop to train the model instead.

```python
 final_loss = slim.learning.train(
        train_op,
        logdir=ckpt_dir,
        number_of_steps=5000,
        graph=graph,
        save_summaries_secs=5,
        log_every_n_steps=500)

```


In [ ]:
# Create fake data to test the mode

def produce_batch(batch_size, noise=0.3):
    xs = np.random.random(size=[batch_size, 1]) * 10
    ys = np.sin(xs) + 5 + np.random.normal(size=[batch_size, 1], scale=noise)
    return [xs.astype(np.float32), ys.astype(np.float32)]

# needs to run under with tf.Graph().as_default():
def convert_data_to_tensors(x, y):
    inputs = tf.constant(x)
    inputs.set_shape([None, 1])
    
    outputs = tf.constant(y)
    outputs.set_shape([None, 1])
    return inputs, outputs

x_train, y_train = produce_batch(200)
x_test, y_test = produce_batch(200)
plt.scatter(x_train, y_train)

I can run and the checkpoints will be save and I can change the learn rate and run again

In [ ]:
# clean up checkppoints
!rm -vf tmp/regression_model/*

In [ ]:
# Train a model
ckpt_dir = 'tmp/regression_model/'

with tf.Graph().as_default():
    
    tf.logging.set_verbosity(tf.logging.INFO)
    
    inputs, targets = convert_data_to_tensors(x_train, y_train)

    # Make the model.
    predictions, nodes = regression_model(inputs, is_training=True)

    # Add the loss function to the graph.
    loss = tf.losses.mean_squared_error(labels=targets, predictions=predictions)
    
    # The total loss is the user's loss plus any regularization losses.
    total_loss =  tf.losses.get_total_loss()# slim.losses.get_total_loss()

    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0075)
    train_op = slim.learning.create_train_op(total_loss, optimizer) 

    # Run the training inside a session.
    final_loss = slim.learning.train(
        train_op,
        logdir=ckpt_dir,
        number_of_steps=5000,
        save_summaries_secs=5,
        log_every_n_steps=500)

print("Finished training. Last batch loss:", final_loss)
print("Checkpoint saved in %s" % ckpt_dir)

In [ ]:
with tf.Graph().as_default():
    
    tf.logging.set_verbosity(tf.logging.INFO)
    
    inputs, targets = convert_data_to_tensors(x_train, y_train)

    # Make the model.
    predictions, nodes = regression_model(inputs, is_training=True)

    # Add the loss function to the graph.
    loss = tf.losses.mean_squared_error(labels=targets, predictions=predictions)
    
    # The total loss is the user's loss plus any regularization losses.
    total_loss =  tf.losses.get_total_loss()# slim.losses.get_total_loss()

    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
    train_op = slim.learning.create_train_op(total_loss, optimizer) 

    # Run the training inside a session.
    final_loss = slim.learning.train(
        train_op,
        logdir=ckpt_dir,
        number_of_steps=5000 + 5000,
        save_summaries_secs=5,
        log_every_n_steps=500)

print("Finished training. Last batch loss:", final_loss)
print("Checkpoint saved in %s" % ckpt_dir)

In [ ]:
!ls tmp/regression_model

##### Using slim to define layers and for loop for trainning

Google authors of vggish modesl did that


In [ ]:
x_train, y_train = produce_batch(200)
x_test, y_test = produce_batch(200)
plt.scatter(x_train, y_train)

In [ ]:
def _get_examples_batch():
    
    chosen_id = np.random.randint(0,200, 200)
  
    return x_train[chosen_id], y_train[chosen_id]

In [ ]:
ckpt_dir = 'tmp/regression_model2/'

with tf.Graph().as_default(), tf.Session() as sess:
    
    tf.logging.set_verbosity(tf.logging.INFO)
    
    with tf.variable_scope('mymodel'):
    
        #inputs, targets = convert_data_to_tensors(x_train, y_train)
        inputs = tf.placeholder(tf.float32, shape=(None, 1), name='inputs')
        targets = tf.placeholder(tf.float32, shape=(None, 1), name='targets')
        
        # Make the model.
        predictions, nodes = regression_model(inputs, is_training=True)

        with tf.variable_scope('train'):
            
            # You need this to save the model
            global_step = tf.Variable(0, name='global_step', 
                                      trainable=False,
                                    collections=[tf.GraphKeys.GLOBAL_VARIABLES,
                                     tf.GraphKeys.GLOBAL_STEP])
            
            # Add the loss function to the graph.
            loss = tf.losses.mean_squared_error(labels=targets, predictions=predictions)
    
            # The total loss is the user's loss plus any regularization losses.
            total_loss =  tf.losses.get_total_loss()# slim.losses.get_total_loss()
            
            # Specify the optimizer and create the train op:
            optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)    
            optimizer.minimize(total_loss,global_step=global_step, name='train_op')
            
            train_op = sess.graph.get_operation_by_name('mymodel/train/train_op')
            
            targets_tensor = sess.graph.get_tensor_by_name('mymodel/targets:0')
            inputs_tensor = sess.graph.get_tensor_by_name('mymodel/inputs:0')
            global_step_tensor = sess.graph.get_tensor_by_name('mymodel/train/global_step:0')

    # Initialize all variables in the model, and then load the pre-trained
    sess.run(tf.global_variables_initializer())
    minibatch_size = 50
    n_batches = 20000
    costs= []
    for _num_batches in range(n_batches):
                    
        # Dynamic creates systehetic data
        (xt, yt) = _get_examples_batch()
        
        for i in range(0, 200, minibatch_size):
        
            # Get pair of (X, y) of the current minibatch/chunk
            x_mini = xt[i:i + minibatch_size]
            y_mini = yt[i:i + minibatch_size]

            [loss, num_steps, _] = sess.run( [ total_loss, train_op, global_step_tensor],
                                  feed_dict={inputs_tensor: x_mini, targets_tensor: y_mini })
        
        if _num_batches % 500 == 0:
            costs.append(loss)
            print("Epoch {}/{} loss: {}".format(_num_batches + 1,n_batches, loss ))
    
    
    # Prediction
    #inputs, targets = convert_data_to_tensors(x_test, y_test)
  
    # Create the model structure. (Parameters will be loaded below.)
    predictions, end_points = regression_model(inputs, is_training=False)

#     # Make a session which restores the old parameters from a checkpoint.
#     inputs, predictions, targets = sess.run([inputs, predictions, targets],
#                                            feed_dict={inputs_tensor: x_test, targets_tensor: y_test })

    

    
plt.plot(costs, c='b')
plt.plot(np.ones(len(costs))*0.12,c='r')

# plt.scatter(inputs, targets, c='r');
# plt.scatter(inputs, predictions, c='b');
# plt.title('red=true, blue=predicted')
    
#     # Save model to disk.
#     saver = tf.train.Saver()
#     save_path = saver.save(sess,ckpt_dir, global_step = _num_batches)
#     print("Model saved to {}".format(save_path))

#### Test

In [ ]:
# do a prediction

with tf.Graph().as_default():
    inputs, targets = convert_data_to_tensors(x_test, y_test)
  
    # Create the model structure. (Parameters will be loaded below.)
    predictions, end_points = regression_model(inputs, is_training=False)

    # Make a session which restores the old parameters from a checkpoint.
    sv = tf.train.Supervisor(logdir=ckpt_dir)
    with sv.managed_session() as sess:
        inputs, predictions, targets = sess.run([inputs, predictions, targets])

plt.scatter(inputs, targets, c='r');
plt.scatter(inputs, predictions, c='b');
plt.title('red=true, blue=predicted')

#### Evaluation model

TODO: NOT working Is hard or complicated to work with

You have to run the script in parallel

In [ ]:
log_dir = 'tmp/log'

# Choose the metrics to compute:
names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
    'RMSE Linear': slim.metrics.streaming_mean_squared_error(predictions=predictions, labels=targets),
    'Mean Abs Error': slim.metrics.streaming_mean_absolute_error(predictions, targets)
})


# Create the summary ops such that they also print out to std output:
summary_ops = []
for metric_name, metric_value in names_to_values.items():
     
    op = tf.summary.scalar(metric_name, metric_value)
    op = tf.Print(op, [metric_value], metric_name)
    summary_ops.append(op)

num_examples = 10000
batch_size = 32
num_batches = math.ceil(num_examples / float(batch_size))

# Setup the global step.
slim.get_or_create_global_step()

output_dir = 'tmp/regression_model/' # Where the summaries are stored.
eval_interval_secs = 0.5 # How often to run the evaluation.

slim.evaluation.evaluation_loop(
    'local',
    ckpt_dir,
    log_dir,
    num_evals=num_batches,
    eval_op=names_to_updates.values(),
    summary_op=tf.summary.merge(summary_ops),
    eval_interval_secs=eval_interval_secs)

### Convolutional (TODO)

In [ ]:
from datasets import flowers

In [ ]:
def my_cnn(images, num_classes, is_training):  # is_training is not used...
    
    with slim.arg_scope([slim.max_pool2d], kernel_size=[3, 3], stride=2):
        net = slim.conv2d(images, 64, [5, 5])
        net = slim.max_pool2d(net)
        net = slim.conv2d(net, 64, [5, 5])
        net = slim.max_pool2d(net)
        net = slim.flatten(net)
        net = slim.fully_connected(net, 192)
        net = slim.fully_connected(net, num_classes, activation_fn=None)       
    return net


**Apply the model to some randomly generated images.**

In [ ]:
import tensorflow as tf

with tf.Graph().as_default():
    # The model can handle any input size because the first layer is convolutional.
    # The size of the model is determined when image_node is first passed into the my_cnn function.
    # Once the variables are initialized, the size of all the weight matrices is fixed.
    # Because of the fully connected layers, this means that all subsequent images must have the same
    # input size as the first image.
    batch_size, height, width, channels = 3, 28, 28, 3
    images = tf.random_uniform([batch_size, height, width, channels], maxval=1)
    
    # Create the model.
    num_classes = 10
    logits = my_cnn(images, num_classes, is_training=True)
    probabilities = tf.nn.softmax(logits)
  
    # Initialize all the variables (including parameters) randomly.
    init_op = tf.global_variables_initializer()
  
    with tf.Session() as sess:
        # Run the init_op, evaluate the model outputs and print the results:
        sess.run(init_op)
        probabilities = sess.run(probabilities)
        
print('Probabilities Shape:')
print(probabilities.shape)  # batch_size x num_classes 

print('\nProbabilities:')
print(probabilities)

print('\nSumming across all classes (Should equal 1):')
print(np.sum(probabilities, 1)) # Each row sums to 1